<a href="https://colab.research.google.com/github/Moe-phantom/Medical_AI_agent/blob/main/Medical_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq chromadb sentence-transformers numpy


In [ ]:
import os
import json
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum
import numpy as np

# Groq API
from groq import Groq

# Vector DB for RAG
import chromadb
from chromadb.utils import embedding_functions


API KEY =

In [ ]:
# Set your Groq API key
GROQ_API_KEY = "gsk"
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

# Model configuration
MODEL_NAME = "llama-3.3-70b-versatile"


In [ ]:
# ============================================================================
# META SYSTEM PROMPT
# ============================================================================

META_SYSTEM_PROMPT = {
    "role": "Medical Research Assistant",
    "goals": [
        "Provide accurate, evidence-based medical information",
        "Ensure patient safety by including appropriate disclaimers",
        "Retrieve and synthesize information from reliable sources",
        "Maintain professional and empathetic communication"
    ],
    "constraints": [
        "Never provide specific medical diagnoses",
        "Always recommend consulting healthcare professionals",
        "Refuse to provide information on illegal substances or harmful practices",
        "Validate all inputs for safety concerns",
        "Flag any potentially harmful queries"
    ],
    "safety_protocols": [
        "Input validation and sanitization",
        "Maker-Checker loop for all responses",
        "Output safety filtering",
        "Harmful content detection",
        "Mandatory medical disclaimers"
    ]
}

In [ ]:
SYSTEM_PROMPT = """You are a Medical Research Assistant with the following guidelines:

ROLE: Provide accurate, evidence-based medical information to help users understand health topics.

GOALS:
- Give clear, accurate medical information based on current research
- Include appropriate disclaimers and safety warnings
- Cite sources when providing medical facts
- Maintain professional and empathetic tone

CONSTRAINTS:
- NEVER diagnose medical conditions
- NEVER prescribe medications
- NEVER provide advice on illegal substances
- ALWAYS recommend consulting healthcare professionals for medical decisions
- ALWAYS include medical disclaimers

SAFETY:
- Refuse harmful requests (self-harm, illegal activities, etc.)
- Flag queries with concerning intent
- Provide crisis resources when appropriate
"""



In [ ]:
# ============================================================================
# DATA CLASSES
# ============================================================================

class SafetyLevel(Enum):
    SAFE = "safe"
    WARNING = "warning"
    BLOCKED = "blocked"

@dataclass
class ValidationResult:
    valid: bool
    safety_level: SafetyLevel
    sanitized_query: str
    issues: List[str]
    reason: Optional[str] = None

@dataclass
class Document:
    content: str
    metadata: Dict
    relevance_score: float

@dataclass
class Response:
    answer: str
    confidence: float
    sources: List[Dict]
    safety_checks: Dict[str, bool]
    needs_refinement: bool
    issues: List[str]

In [ ]:
# ============================================================================
# KNOWLEDGE BASE SETUP
# ============================================================================

# Medical knowledge base
MEDICAL_KNOWLEDGE = [
    {
        "id": "diabetes_001",
        "content": """Type 2 diabetes is a chronic metabolic disorder characterized by high blood sugar levels
        due to insulin resistance. Management strategies include: lifestyle modifications (diet and exercise),
        oral medications (metformin as first-line), GLP-1 receptor agonists, SGLT2 inhibitors, and insulin therapy
        when needed. Regular HbA1c monitoring (target <7% for most patients) is essential. Recent studies show
        that early intensive glucose control can reduce long-term complications.""",
        "topic": "diabetes",
        "source": "ADA Clinical Practice Guidelines 2024",
        "reliability": 0.95
    },
    {
        "id": "hypertension_001",
        "content": """Hypertension is defined as blood pressure ≥140/90 mmHg on multiple readings. First-line
        treatments include ACE inhibitors, ARBs, thiazide diuretics, and calcium channel blockers. Lifestyle
        modifications are crucial: DASH diet, sodium restriction (<2.3g/day), regular exercise (150 min/week),
        weight loss if overweight, and limiting alcohol. Target BP is typically <130/80 mmHg for most adults.
        Home blood pressure monitoring is recommended for diagnosis and management.""",
        "topic": "hypertension",
        "source": "AHA/ACC Hypertension Guidelines 2024",
        "reliability": 0.97
    },
    {
        "id": "covid_001",
        "content": """COVID-19, caused by SARS-CoV-2 virus, remains a significant public health concern.
        Prevention strategies include vaccination (updated vaccines available), masking in high-risk settings,
        hand hygiene, and ventilation. Treatment varies by severity: mild cases require supportive care,
        high-risk patients may benefit from Paxlovid (nirmatrelvir/ritonavir) if started within 5 days,
        severe cases may require hospitalization with oxygen, steroids, and antivirals. Long COVID affects
        10-30% of infected individuals.""",
        "topic": "covid-19",
        "source": "WHO COVID-19 Guidelines 2024",
        "reliability": 0.96
    },
    {
        "id": "antibiotics_001",
        "content": """Antibiotics are medications that treat bacterial infections by killing bacteria or
        preventing their growth. Common classes include: penicillins (e.g., amoxicillin), cephalosporins,
        macrolides (e.g., azithromycin), fluoroquinolones. Antibiotic resistance is a critical global health
        threat. Best practices: complete full course as prescribed, never share antibiotics, don't use for
        viral infections, culture-guided therapy when possible. Side effects may include GI upset, allergic
        reactions, and opportunistic infections.""",
        "topic": "antibiotics",
        "source": "CDC Antibiotic Guidelines 2024",
        "reliability": 0.94
    },
    {
        "id": "cancer_screening_001",
        "content": """Cancer screening aims to detect cancer early when treatment is most effective.
        Recommended screenings: mammography for breast cancer (age 40-50+), colonoscopy for colorectal
        cancer (age 45+), Pap smear and HPV testing for cervical cancer (age 21-65), low-dose CT for
        lung cancer in high-risk individuals (age 50-80, smoking history). PSA testing for prostate cancer
        is individualized. Early detection significantly improves survival rates across most cancer types.""",
        "topic": "cancer screening",
        "source": "USPSTF Cancer Screening Guidelines 2024",
        "reliability": 0.96
    },
    {
        "id": "mental_health_001",
        "content": """Depression and anxiety are common mental health conditions affecting millions worldwide.
        Treatment approaches include: psychotherapy (CBT, DBT), medications (SSRIs, SNRIs), lifestyle
        modifications (exercise, sleep hygiene, stress management), and social support. Early intervention
        improves outcomes. Crisis resources: National Suicide Prevention Lifeline (988), Crisis Text Line
        (text HOME to 741741). Combination therapy (medication + psychotherapy) often more effective than
        either alone.""",
        "topic": "mental health",
        "source": "APA Clinical Practice Guidelines 2024",
        "reliability": 0.95
    },
    {
        "id": "nutrition_001",
        "content": """Evidence-based nutrition principles for chronic disease prevention: Mediterranean diet
        shows benefits for cardiovascular health, plant-based diets may reduce cancer risk, adequate fiber
        intake (25-30g/day) improves digestive health. Key nutrients: omega-3 fatty acids, vitamin D,
        calcium, iron. Limit processed foods, added sugars (<10% calories), and saturated fats. Stay
        hydrated (8-10 cups water/day). Individual needs vary by age, sex, activity level, and health
        conditions. Consult registered dietitian for personalized advice.""",
        "topic": "nutrition",
        "source": "Academy of Nutrition and Dietetics 2024",
        "reliability": 0.93
    },
    {
        "id": "vaccines_001",
        "content": """Vaccination is one of the most effective public health interventions. Recommended adult
        vaccines include: annual influenza vaccine, Tdap booster every 10 years, shingles vaccine (age 50+),
        pneumococcal vaccines (age 65+ or high-risk), COVID-19 vaccines (updated formulations), HPV vaccine
        (up to age 26, sometimes 45). Vaccines undergo rigorous safety testing. Benefits far outweigh risks
        for vast majority of people. Some vaccines contraindicated in immunocompromised individuals or during
        pregnancy - consult healthcare provider.""",
        "topic": "vaccines",
        "source": "CDC Immunization Guidelines 2024",
        "reliability": 0.98
    }
]

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

# ============================================================================
# VECTOR DATABASE INITIALIZATION
# ============================================================================

def initialize_vector_db():
    """Initialize ChromaDB with medical knowledge base"""
    print("🔧 Initializing Vector Database...")

    # Create ChromaDB client
    chroma_client = chromadb.Client()

    # Create collection with sentence transformer embeddings
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="all-MiniLM-L6-v2"
    )

    collection = chroma_client.create_collection(
        name="medical_knowledge",
        embedding_function=sentence_transformer_ef,
        metadata={"description": "Medical research knowledge base"},
        get_or_create=True # Add this line to get the collection if it already exists
    )

    # Add documents to collection
    # Only add documents if the collection is newly created or empty
    if collection.count() == 0:
        collection.add(
            documents=[doc["content"] for doc in MEDICAL_KNOWLEDGE],
            metadatas=[{
                "source": doc["source"],
                "topic": doc["topic"],
                "reliability": doc["reliability"]
            } for doc in MEDICAL_KNOWLEDGE],
            ids=[doc["id"] for doc in MEDICAL_KNOWLEDGE]
        )
        print(f"✅ Vector database initialized with {len(MEDICAL_KNOWLEDGE)} documents")
    else:
        print(f"✅ Vector database collection 'medical_knowledge' already exists with {collection.count()} documents.")
        print("ℹ️ Skipping document addition as collection is not empty.")

    return collection

# Initialize the database
vector_db = initialize_vector_db()


🔧 Initializing Vector Database...
✅ Vector database collection 'medical_knowledge' already exists with 8 documents.
ℹ️ Skipping document addition as collection is not empty.


In [ ]:
# ============================================================================
# SAFETY LAYER 1: INPUT VALIDATION
# ============================================================================

class InputValidator:
    """Validates and sanitizes user inputs"""

    # Harmful patterns to detect
    MALICIOUS_PATTERNS = [
        r'<script.*?>.*?</script>',  # XSS
        r'javascript:',
        r'onerror\s*=',
        r'union.*select',  # SQL injection
        r'drop.*table',
        r'insert.*into',
        r'\.\./|\.\.\\',  # Path traversal
    ]

    # Harmful intent keywords
    HARMFUL_KEYWORDS = [
        'how to harm', 'how to kill', 'suicide methods', 'self harm',
        'overdose intentionally', 'poison someone', 'make drugs',
        'synthesize illegal', 'bomb making', 'weapon'
    ]

    # Crisis keywords (need special handling)
    CRISIS_KEYWORDS = [
        'want to die', 'kill myself', 'end my life', 'suicidal'
    ]

    @staticmethod
    def validate(query: str) -> ValidationResult:
        """Validate and sanitize input query"""
        print("\n🔍 SAFETY LAYER 1: Input Validation")
        print("=" * 60)

        issues = []

        # Check for empty input
        if not query or not query.strip():
            print("❌ Validation failed: Empty input")
            return ValidationResult(
                valid=False,
                safety_level=SafetyLevel.BLOCKED,
                sanitized_query="",
                issues=["Empty query"],
                reason="Query cannot be empty"
            )

        sanitized = query.strip()

        # Check length
        if len(sanitized) > 1000:
            print("❌ Validation failed: Query too long")
            return ValidationResult(
                valid=False,
                safety_level=SafetyLevel.BLOCKED,
                sanitized_query="",
                issues=["Query too long"],
                reason="Query exceeds maximum length of 1000 characters"
            )

        # Check for malicious patterns
        query_lower = sanitized.lower()
        for pattern in InputValidator.MALICIOUS_PATTERNS:
            if re.search(pattern, query_lower, re.IGNORECASE):
                print(f"❌ Validation failed: Malicious pattern detected: {pattern}")
                return ValidationResult(
                    valid=False,
                    safety_level=SafetyLevel.BLOCKED,
                    sanitized_query="",
                    issues=[f"Malicious pattern: {pattern}"],
                    reason="Input contains potentially malicious content"
                )

        # Check for crisis keywords
        for keyword in InputValidator.CRISIS_KEYWORDS:
            if keyword in query_lower:
                print(f"⚠️  Crisis keyword detected: {keyword}")
                return ValidationResult(
                    valid=False,
                    safety_level=SafetyLevel.BLOCKED,
                    sanitized_query=sanitized,
                    issues=[f"Crisis keyword: {keyword}"],
                    reason="""I'm concerned about your wellbeing. If you're having thoughts of suicide,
                    please reach out for immediate help:

                    🆘 National Suicide Prevention Lifeline: 988 (US)
                    🆘 Crisis Text Line: Text HOME to 741741
                    🆘 International: https://findahelpline.com

                    Please talk to a mental health professional or trusted person right away."""
                )

        # Check for harmful intent
        for keyword in InputValidator.HARMFUL_KEYWORDS:
            if keyword in query_lower:
                print(f"❌ Validation failed: Harmful intent detected: {keyword}")
                return ValidationResult(
                    valid=False,
                    safety_level=SafetyLevel.BLOCKED,
                    sanitized_query="",
                    issues=[f"Harmful keyword: {keyword}"],
                    reason="This query appears to request harmful information. I cannot provide this information."
                )

        print("✅ Input validation passed")
        return ValidationResult(
            valid=True,
            safety_level=SafetyLevel.SAFE,
            sanitized_query=sanitized,
            issues=[]
        )


In [ ]:
# ============================================================================
# AGENTIC RAG: DOCUMENT RETRIEVAL
# ============================================================================

class DocumentRetriever:
    """Retrieves relevant documents from vector database"""

    def __init__(self, collection):
        self.collection = collection

    def retrieve(self, query: str, top_k: int = 3) -> List[Document]:
        """Retrieve top-k relevant documents"""
        print("\n📚 RETRIEVAL: Searching Knowledge Base")
        print("=" * 60)

        # Query vector database
        results = self.collection.query(
            query_texts=[query],
            n_results=top_k
        )

        documents = []
        if results['documents'] and results['documents'][0]:
            for i, (doc, metadata, distance) in enumerate(zip(
                results['documents'][0],
                results['metadatas'][0],
                results['distances'][0]
            )):
                # Convert distance to similarity score (lower distance = higher similarity)
                relevance_score = 1 / (1 + distance)

                documents.append(Document(
                    content=doc,
                    metadata=metadata,
                    relevance_score=relevance_score
                ))

                print(f"📄 Document {i+1}:")
                print(f"   Topic: {metadata.get('topic', 'N/A')}")
                print(f"   Source: {metadata.get('source', 'N/A')}")
                print(f"   Relevance: {relevance_score:.2f}")

        print(f"\n✅ Retrieved {len(documents)} relevant documents")
        return documents



In [ ]:
# ============================================================================
# MAKER AGENT: GENERATE INITIAL RESPONSE
# ============================================================================

class MakerAgent:
    """Generates initial response using Groq API"""

    def __init__(self, client, model_name):
        self.client = client
        self.model = model_name

    def generate(self, query: str, documents: List[Document]) -> Response:
        """Generate initial response"""
        print("\n🤖 MAKER AGENT: Generating Response")
        print("=" * 60)

        if not documents:
            print("⚠️  No relevant documents found")
            return Response(
                answer="I don't have specific information about this topic in my knowledge base. Please consult with a healthcare professional for accurate medical advice.",
                confidence=0.3,
                sources=[],
                safety_checks={},
                needs_refinement=True,
                issues=["No relevant documents"]
            )

        # Prepare context from retrieved documents
        context = "\n\n".join([
            f"Source: {doc.metadata.get('source', 'Unknown')}\n{doc.content}"
            for doc in documents
        ])

        # Create prompt for Groq
        prompt = f"""Based on the following medical research information, answer the user's question.

CONTEXT:
{context}

USER QUESTION: {query}

INSTRUCTIONS:
1. Provide accurate information based on the context
2. Include relevant sources
3. Add appropriate medical disclaimers
4. Be clear and concise
5. If context is insufficient, acknowledge limitations

ANSWER:"""

        try:
            # Call Groq API
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=1000
            )

            answer = response.choices[0].message.content

            # Extract sources
            sources = [
                {
                    "source": doc.metadata.get('source', 'Unknown'),
                    "reliability": doc.metadata.get('reliability', 0.8),
                    "relevance": doc.relevance_score
                }
                for doc in documents
            ]

            print("✅ Initial response generated")
            print(f"📊 Response length: {len(answer)} characters")

            return Response(
                answer=answer,
                confidence=documents[0].metadata.get('reliability', 0.8),
                sources=sources,
                safety_checks={},
                needs_refinement=False,
                issues=[]
            )

        except Exception as e:
            print(f"❌ Error generating response: {str(e)}")
            return Response(
                answer="I encountered an error generating a response. Please try again.",
                confidence=0.0,
                sources=[],
                safety_checks={},
                needs_refinement=True,
                issues=[f"Generation error: {str(e)}"]
            )


In [ ]:
# ============================================================================
# CHECKER AGENT: VALIDATE RESPONSE
# ============================================================================

class CheckerAgent:
    """Reviews response for safety and correctness"""

    REQUIRED_DISCLAIMER_PHRASES = [
        'consult', 'healthcare professional', 'doctor', 'medical advice',
        'physician', 'healthcare provider'
    ]

    DIAGNOSIS_TERMS = [
        'you have', 'you are diagnosed', 'you suffer from',
        'you definitely have', 'your condition is'
    ]

    HARMFUL_CONTENT = [
        'self-harm', 'suicide', 'illegal drug', 'recreational drug synthesis'
    ]

    def check(self, response: Response, query: str) -> Response:
        """Perform safety checks on response"""
        print("\n🔍 CHECKER AGENT: Validating Response")
        print("=" * 60)

        issues = []
        checks = {
            'has_disclaimer': False,
            'no_diagnosis': True,
            'has_sources': False,
            'appropriate_length': False,
            'no_harmful_content': True,
            'confidence_reasonable': False
        }

        answer_lower = response.answer.lower()

        # Check 1: Has disclaimer
        has_disclaimer = any(phrase in answer_lower for phrase in self.REQUIRED_DISCLAIMER_PHRASES)
        checks['has_disclaimer'] = has_disclaimer
        if has_disclaimer:
            print("✅ CHECK: Medical disclaimer present")
        else:
            print("⚠️  CHECK: Missing medical disclaimer")
            issues.append("Missing medical disclaimer")

        # Check 2: No direct diagnosis
        has_diagnosis = any(term in answer_lower for term in self.DIAGNOSIS_TERMS)
        checks['no_diagnosis'] = not has_diagnosis
        if not has_diagnosis:
            print("✅ CHECK: No inappropriate diagnosis language")
        else:
            print("❌ CHECK: Contains direct diagnosis language")
            issues.append("Contains inappropriate diagnosis")

        # Check 3: Has sources
        checks['has_sources'] = len(response.sources) > 0
        if checks['has_sources']:
            print(f"✅ CHECK: {len(response.sources)} sources cited")
        else:
            print("⚠️  CHECK: No sources cited")
            issues.append("No sources cited")

        # Check 4: Appropriate length
        length_ok = 100 < len(response.answer) < 2000
        checks['appropriate_length'] = length_ok
        if length_ok:
            print(f"✅ CHECK: Appropriate length ({len(response.answer)} chars)")
        else:
            print(f"⚠️  CHECK: Length issue ({len(response.answer)} chars)")
            issues.append("Response length inappropriate")

        # Check 5: No harmful content
        has_harmful = any(term in answer_lower for term in self.HARMFUL_CONTENT)
        checks['no_harmful_content'] = not has_harmful
        if not has_harmful:
            print("✅ CHECK: No harmful content detected")
        else:
            print("❌ CHECK: Harmful content detected")
            issues.append("Harmful content detected")

        # Check 6: Confidence reasonable
        checks['confidence_reasonable'] = 0.5 <= response.confidence <= 1.0
        if checks['confidence_reasonable']:
            print(f"✅ CHECK: Confidence reasonable ({response.confidence:.2f})")
        else:
            print(f"⚠️  CHECK: Confidence issue ({response.confidence:.2f})")
            issues.append("Confidence score unusual")

        # Determine if refinement needed
        needs_refinement = len(issues) > 0 and checks['no_harmful_content']

        if not checks['no_harmful_content']:
            print("\n❌ CRITICAL: Response contains harmful content - BLOCKED")
        elif needs_refinement:
            print(f"\n⚠️  Response needs refinement ({len(issues)} issues)")
        else:
            print("\n✅ All checks passed - Response approved")

        response.safety_checks = checks
        response.needs_refinement = needs_refinement
        response.issues = issues

        return response



In [ ]:
# ============================================================================
# REFINER AGENT: IMPROVE RESPONSE
# ============================================================================

class RefinerAgent:
    """Refines response based on checker feedback"""

    def __init__(self, client, model_name):
        self.client = client
        self.model = model_name

    def refine(self, response: Response, query: str) -> Response:
        """Refine response to address issues"""
        print("\n🔧 REFINER AGENT: Improving Response")
        print("=" * 60)

        if not response.needs_refinement:
            print("ℹ️  No refinement needed")
            return response

        # Build refinement instructions
        instructions = []

        if not response.safety_checks.get('has_disclaimer'):
            instructions.append("Add a clear medical disclaimer advising to consult healthcare professionals")

        if not response.safety_checks.get('has_sources'):
            instructions.append("Include citations to the provided sources")

        if not response.safety_checks.get('no_diagnosis'):
            instructions.append("Remove any direct diagnosis statements - use general information only")

        if not response.safety_checks.get('appropriate_length'):
            if len(response.answer) < 100:
                instructions.append("Expand the response with more detail")
            else:
                instructions.append("Make the response more concise")

        refinement_prompt = f"""The following response needs improvement:

ORIGINAL RESPONSE:
{response.answer}

ISSUES TO ADDRESS:
{chr(10).join(f'- {issue}' for issue in instructions)}

USER'S ORIGINAL QUESTION: {query}

Please provide an improved response that addresses all the issues while maintaining accuracy and helpfulness.

IMPROVED RESPONSE:"""

        try:
            refined = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": refinement_prompt}
                ],
                temperature=0.2,
                max_tokens=1000
            )

            response.answer = refined.choices[0].message.content
            print("✅ Response refined successfully")

        except Exception as e:
            print(f"⚠️  Refinement error: {str(e)}")
            # Fallback: manually add disclaimer
            if not response.safety_checks.get('has_disclaimer'):
                response.answer += "\n\n⚠️ IMPORTANT: This information is for educational purposes only. Please consult with a qualified healthcare provider for personalized medical advice, diagnosis, or treatment."

            if response.sources and not response.safety_checks.get('has_sources'):
                response.answer += "\n\nSources:\n" + "\n".join([
                    f"- {s['source']} (Reliability: {s['reliability']*100:.0f}%)"
                    for s in response.sources
                ])

        return response


In [ ]:
# ============================================================================
# MAIN AGENTIC RAG PIPELINE
# ============================================================================

class AgenticRAGSystem:
    """Main orchestrator for the agentic RAG system"""

    def __init__(self):
        self.validator = InputValidator()
        self.retriever = DocumentRetriever(vector_db)
        self.maker = MakerAgent(client, MODEL_NAME)
        self.checker = CheckerAgent()
        self.refiner = RefinerAgent(client, MODEL_NAME)

    def process_query(self, query: str) -> Dict:
        """Process user query through the full pipeline"""
        print("\n" + "=" * 80)
        print("🚀 AGENTIC RAG PIPELINE STARTED")
        print("=" * 80)
        print(f"Query: {query}")

        # Step 1: Validate input
        validation = self.validator.validate(query)
        if not validation.valid:
            return {
                'success': False,
                'stage': 'validation',
                'error': validation.reason,
                'issues': validation.issues
            }

        # Step 2: Retrieve documents
        documents = self.retriever.retrieve(validation.sanitized_query)

        # Step 3: Generate response (Maker)
        response = self.maker.generate(validation.sanitized_query, documents)

        # Step 4: Check response (Checker)
        response = self.checker.check(response, validation.sanitized_query)

        # Critical safety check - block if harmful
        if not response.safety_checks.get('no_harmful_content', True):
            print("\n🛑 PIPELINE BLOCKED: Harmful content detected")
            return {
                'success': False,
                'stage': 'safety_check',
                'error': 'Response failed safety validation',
                'issues': response.issues
            }

        # Step 5: Refine if needed
        if response.needs_refinement:
            response = self.refiner.refine(response, validation.sanitized_query)
            # Re-check after refinement
            response = self.checker.check(response, validation.sanitized_query)

        print("\n" + "=" * 80)
        print("✅ PIPELINE COMPLETED SUCCESSFULLY")
        print("=" * 80)

        return {
            'success': True,
            'stage': 'complete',
            'answer': response.answer,
            'confidence': response.confidence,
            'sources': response.sources,
            'safety_checks': response.safety_checks,
            'was_refined': response.needs_refinement,
            'num_documents': len(documents)
        }


In [ ]:
# ============================================================================
# USAGE EXAMPLES
# ============================================================================

def run_examples():
    """Run example queries to demonstrate the system"""

    print("\n" + "🎯" * 40)
    print("AGENTIC RAG SYSTEM - EXAMPLE DEMONSTRATIONS")
    print("🎯" * 40)

    rag_system = AgenticRAGSystem()

    # Example 1: Safe medical query
    print("\n\n📋 EXAMPLE 1: Safe Medical Query")
    print("-" * 80)
    result1 = rag_system.process_query("What are the treatment options for type 2 diabetes?")
    if result1['success']:
        print("\n📝 FINAL ANSWER:")
        print(result1['answer'])
        print(f"\n📊 Confidence: {result1['confidence']*100:.0f}%")
        print(f"📚 Documents used: {result1['num_documents']}")
        print(f"🔧 Refined: {result1['was_refined']}")

    # Example 2: Query requiring refinement
    print("\n\n📋 EXAMPLE 2: Query Requiring Refinement")
    print("-" * 80)
    result2 = rag_system.process_query("Tell me about managing high blood pressure")
    if result2['success']:
        print("\n📝 FINAL ANSWER:")
        print(result2['answer'][:500] + "..." if len(result2['answer']) > 500 else result2['answer'])
        print(f"\n🔧 Was refined: {result2['was_refined']}")

    # Example 3: Blocked harmful query
    print("\n\n📋 EXAMPLE 3: Blocked Harmful Query")
    print("-" * 80)
    result3 = rag_system.process_query("How can I overdose on antibiotics?")
    if not result3['success']:
        print(f"\n🛑 Query blocked at stage: {result3['stage']}")
        print(f"❌ Reason: {result3['error']}")

    # Example 4: Edge case - no relevant documents
    print("\n\n📋 EXAMPLE 4: Query with No Relevant Documents")
    print("-" * 80)
    result4 = rag_system.process_query("What is quantum computing?")
    if result4['success']:
        print("\n📝 FINAL ANSWER:")
        print(result4['answer'])
        print(f"\n📊 Confidence: {result4['confidence']*100:.0f}%")


In [ ]:
# ============================================================================
# INTERACTIVE MODE
# ============================================================================

def interactive_mode():
    """Run system in interactive mode"""
    print("\n" + "🏥" * 40)
    print("MEDICAL RESEARCH ASSISTANT - INTERACTIVE MODE")
    print("🏥" * 40)
    print("\nType 'quit' to exit, 'examples' to see demonstrations")

    rag_system = AgenticRAGSystem()

    while True:
        print("\n" + "-" * 80)
        query = input("\n💬 Your question: ").strip()

        if query.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break

        if query.lower() == 'examples':
            run_examples()
            continue

        if not query:
            continue

        result = rag_system.process_query(query)

        if result['success']:
            print("\n" + "=" * 80)
            print("✅ RESPONSE")
            print("=" * 80)
            print(result['answer'])
            print("\n" + "-" * 80)
            print(f"📊 Confidence: {result['confidence']*100:.0f}%")
            print(f"📚 Documents: {result['num_documents']}")
            print(f"🔧 Refined: {'Yes' if result['was_refined'] else 'No'}")
            print(f"✅ Safety Checks: {sum(result['safety_checks'].values())}/{len(result['safety_checks'])}")

            if result['sources']:
                print("\n📖 Sources:")
                for i, source in enumerate(result['sources'], 1):
                    print(f"   {i}. {source['source']} (Reliability: {source['reliability']*100:.0f}%)")
        else:
            print("\n" + "=" * 80)
            print("❌ ERROR")
            print("=" * 80)
            print(f"Stage: {result['stage']}")
            print(f"Error: {result['error']}")
            if result.get('issues'):
                print(f"Issues: {', '.join(result['issues'])}")


In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("""
    ╔══════════════════════════════════════════════════════════════════════╗
    ║                                                                      ║
    ║          AGENTIC RAG SYSTEM WITH SAFETY MEASURES                     ║
    ║          Medical Research Assistant                                  ║
    ║                                                                      ║
    ║  Components:                                                         ║
    ║  ✓ Input Validation Layer                                            ║
    ║  ✓ Vector-based Document Retrieval (ChromaDB)                        ║
    ║  ✓ Maker Agent (Groq API - Response Generation)                      ║
    ║  ✓ Checker Agent (Safety & Quality Validation)                       ║
    ║  ✓ Refiner Agent (Automated Improvement)                             ║
    ║                                                                      ║
    ║  Safety Features:                                                    ║
    ║  ✓ Malicious input detection                                         ║
    ║  ✓ Harmful content filtering                                         ║
    ║  ✓ Crisis intervention routing                                       ║
    ║  ✓ Medical disclaimer enforcement                                    ║
    ║  ✓ Maker-Checker loop with refinement                                ║
    ║                                                                      ║
    ╚══════════════════════════════════════════════════════════════════════╝
    """)

    print("\n⚙️  Choose mode:")
    print("1. Run examples")
    print("2. Interactive mode")

    choice = input("\nEnter choice (1 or 2): ").strip()

    if choice == "1":
        run_examples()
    else:
        interactive_mode()



    ╔══════════════════════════════════════════════════════════════════════╗
    ║                                                                      ║
    ║          AGENTIC RAG SYSTEM WITH SAFETY MEASURES                     ║
    ║          Medical Research Assistant                                  ║
    ║                                                                      ║
    ║  Components:                                                         ║
    ║  ✓ Input Validation Layer                                            ║
    ║  ✓ Vector-based Document Retrieval (ChromaDB)                        ║
    ║  ✓ Maker Agent (Groq API - Response Generation)                      ║
    ║  ✓ Checker Agent (Safety & Quality Validation)                       ║
    ║  ✓ Refiner Agent (Automated Improvement)                             ║
    ║                                                                      ║
    ║  Safety Features:                                                    